In [5]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_thomas import get_thomas_data
from sindy_utils import library_size
from training import train_network
#import tensorflow as tf
import tensorflow.compat.v1 as tf


# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_thomas_data(1024, noise_strength=noise_strength)
validation_data = get_thomas_data(20, noise_strength=noise_strength)

# Set up model and training parameters

In [15]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100
params['print_frequency'] = 1 # for testing

# training time cutoffs
params['max_epochs'] = 5001 # WAY TOO LONG ON AN M1 MAX
params['max_epochs'] = 1000
params['refinement_epochs'] = 1001 # idk what this is 
params['refinement_epochs'] = 100

# Run training experiments

In [16]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2023-01-12 20:38:26.061587: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-12 20:38:26.062467: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-01-12 20:38:26.141118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-12 20:38:26.502273: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-12 20:38:37.113234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 0
   training loss 0.18387821316719055, (0.18386748, 141.15512, 0.017425064, 0.89796865)
   validation loss 0.226305291056633, (0.22629198, 110.724266, 0.043270793, 0.89796865)


2023-01-12 20:38:37.723271: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


decoder loss ratio: 0.483884, decoder SINDy loss  ratio: 3.442894
Epoch 1
   training loss 0.18032748997211456, (0.18031718, 72.07468, 0.017060902, 0.8617107)
   validation loss 0.2154376059770584, (0.21542646, 54.729374, 0.025268234, 0.8617107)
decoder loss ratio: 0.460650, decoder SINDy loss  ratio: 2.010498
Epoch 2
   training loss 0.17431622743606567, (0.17430301, 58.40391, 0.049408995, 0.82658476)
   validation loss 0.20572993159294128, (0.20571606, 53.079967, 0.055963945, 0.82658476)
decoder loss ratio: 0.439886, decoder SINDy loss  ratio: 4.452840
Epoch 3
   training loss 0.10983040928840637, (0.10980943, 206.41997, 0.13243836, 0.7736687)
   validation loss 0.11630702763795853, (0.116287425, 162.3036, 0.11866428, 0.7736687)
decoder loss ratio: 0.248659, decoder SINDy loss  ratio: 9.441669
Epoch 4
   training loss 0.1054980605840683, (0.10548855, 81.902664, 0.018754398, 0.76261073)
   validation loss 0.10532785207033157, (0.105317585, 46.66145, 0.026349248, 0.76261073)
decoder lo